In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from transformers import AutoTokenizer
import pandas as pd
import seaborn as sns
from datasets import Dataset
from os.path import join as oj
import pickle as pkl
import os
import analyze_utils
import sys
import iprompt.data

import torch
from transformers import AutoTokenizer, OPTForCausalLM
from imodelsx import explain_dataset_iprompt, get_add_two_numbers_dataset

In [2]:
tokenizer = AutoTokenizer.from_pretrained("facebook/galactica-6.7b")
model = OPTForCausalLM.from_pretrained("facebook/galactica-6.7b", device_map="auto", torch_dtype=torch.float16)

In [3]:
def gen(model, input_text):
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(input_ids, max_length=200)
    return tokenizer.decode(outputs[0])

In [4]:
gen(model, "[START_I_SMILES]")

'[START_I_SMILES]CC1=CC=C(C=C1)C2=NN=C(N2C3=CC=CC=C3)SCC(=O)NC4=CC=CC=C4OC[END_I_SMILES]\n\n### Molecular Formula\n\nC24H22N4O2S\n\n## Chemical and Physical Properties\n\nThe following are chemical properties for 2-[[4,5-bis(p-tolyl)-1,2,4-triazol-3-yl]sulfanyl]-N-(2-methoxyphenyl)acetamide.\n\n### Computed Properties\n\n| Property Name | Property Value\n| --- | ----------- |\n| Molecular Weight | 430.5\n| XLogP3-AA Log P | 4.9\n| Hydrogen Bond Donor Count | 1\n| Hydrogen Bond Acceptor Count |'

In [9]:
gen(
    model,
    'Here is a SMILES formula:\n[START_I_SMILES]O=C(O)CCCC1=CC=C(N(CCCl)CCCl)C=C1[END_I_SMILES]\nQuestion: Will the chemical compound penetrate the blood-brain barrier?\nAnswer:'
)

'Here is a SMILES formula:\n[START_I_SMILES]O=C(O)CCCC1=CC=C(N(CCCl)CCCl)C=C1[END_I_SMILES]\nQuestion: Will the chemical compound penetrate the blood-brain barrier?\nAnswer: Yes</s>'

# Example find prompt

In [10]:
# get a simple dataset of adding two numbers
input_strings, output_strings = get_add_two_numbers_dataset(num_examples=100)
for i in range(5):
    print(repr(input_strings[i]), repr(output_strings[i]))

# explain the relationship between the inputs and outputs
# with a natural-language prompt string
prompts, metadata = explain_dataset_iprompt(
    input_strings=input_strings,
    output_strings=output_strings,
    checkpoint='EleutherAI/gpt-j-6B', # which language model to use
    num_learned_tokens=3, # how long of a prompt to learn
    n_shots=3, # shots per example

    n_epochs=15, # how many epochs to search
    verbose=0, # how much to print
    llm_float16=True, # whether to load the model in float_16
)

'Given the input numbers 9 and 4, the answer is' ' 13.\n\n'
'Given the input numbers 6 and 6, the answer is' ' 12.\n\n'
'Given the input numbers 5 and 4, the answer is' ' 9.\n\n'
'Given the input numbers 2 and 4, the answer is' ' 6.\n\n'
'Given the input numbers 7 and 0, the answer is' ' 7.\n\n'
loading model...
Training with 19 possible answers / random acc 5.3% / majority acc 10.3%
Beginning epoch 0


  0%|          | 0/157 [00:00<?, ?it/s]

population_pool: [' A computer only', ' The problem with', ' [Click on', ' How are these', ' When I was', ' Given you have', ' When I was', ' How are these', ' A computer only', ' A computer only', ' A computer only', ' The problem with']


Loss = 3.055:   1%|          | 1/157 [00:11<29:24, 11.31s/it]

population_pool: [' How to add', ' How to add', ' A simple function', ' Given an integer', ' The given number', ' The given number', ' (Input)', ' (Input)', ' When it says', ' [5]:', ' What does this', ' What does this']


...

population_pool: [' Add two integers', ' Create a number', ' Write an integer', '  The sum', ' Find a sum', ' The sum number', ' Convert an integer', ' A simple multiplication', ' Read two integers', ' "5 +', ' Input two integer', ' How to add']


Loss = 1.340:  46%|████▌     | 72/157 [12:54<15:15, 10.77s/it]

population_pool: [' Add two integers', ' Create a number', ' Write an integer', '  The sum', ' Find a sum', ' The sum number', ' Convert an integer', ' A simple multiplication', ' Read two integers', ' "5 +', ' Input two integer', ' How to add']


Loss = 1.340:  46%|████▌     | 72/157 [12:55<15:15, 10.77s/it]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_618572/2216990027.py:10 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_618572/2216990027.py'                       │
│                                                                                                  │
│ /home/chansingh/imodelsX/imodelsx/iprompt/api.py:457 in explain_dataset_iprompt                  │
│                                                                                                  │
│   454 │   torch.manual_seed(seed)                                                                │
│   455 │   random.seed(seed)                                                                      │
│   456 │   r = defaultdict(list)                                                                  │
│ ❱ 457 │   r = train_model(                                                                       │
│   458 │   │   r=r,                                                                               │
│   459 │   │   input_strs=input_strings,                                                          │
│   460 │   │   output_strs=output_strings,                                                        │
│                                                                                                  │
│ /home/chansingh/imodelsX/imodelsx/iprompt/api.py:178 in train_model                              │
│                                                                                                  │
│   175 │   │   │   y_tokenized = tok(y_text).to(device)                                           │
│   176 │   │   │   full_text_tokenized = tok(batch['text']).to(device)                            │
│   177 │   │   │                                                                                  │
│ ❱ 178 │   │   │   loss, n_correct = model.compute_loss_and_call_backward(                        │
│   179 │   │   │   │   x_tokenized=x_tokenized,                                                   │
│   180 │   │   │   │   y_tokenized=y_tokenized,                                                   │
│   181 │   │   │   │   possible_answer_mask=possible_answer_mask,                                 │
│                                                                                                  │
│ /home/chansingh/imodelsX/imodelsx/iprompt/ipromptx.py:328 in compute_loss_and_call_backward      │
│                                                                                                  │
│   325 │   │   population_input_ids = self._get_population_and_random_generations(                │
│   326 │   │   │   full_text_ids=full_text_ids,                                                   │
│   327 │   │   )                                                                                  │
│ ❱ 328 │   │   mutated_population_input_ids = self._mutate(                                       │
│   329 │   │   │   population_input_ids=population_input_ids, full_text_ids=full_text_ids         │
│   330 │   │   )                                                                                  │
│   331 │   │   full_population_input_ids = torch.cat(                                             │
│                                                                                                  │
│ /home/chansingh/imodelsX/imodelsx/iprompt/ipromptx.py:236 in _mutate                             │
│                                                                                                  │
│   233 │   │   │   (random_full_text_ids, truncated_input_ids), dim=1)                            │
│   234 │   │                                                                                      │
│   235 │   │   num_conditional_tokens = full_text_ids.shape[